In [1]:
import numpy as np
import pandas as pd
import time

np.random.seed(2024)

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

from TopoTree import TopoTree, plot_hierarchical_treemap
from TopoMap import TopoMap
from TopoMapMLpack import TopoMapMlPack
from HierarchicalTopoMap import HierarchicalTopoMap, HierarchicalTopoMap2

In [2]:
place = 'TimesSquare'

X_times = np.loadtxt(f'data/topomap-data/{place}.txt',
                     delimiter=",")
X_times.shape

(17520, 6)

In [3]:
start_time = time.time()
topomap_times = TopoMap(X_times)
proj_topomap_times = topomap_times.run()
print("--- %s seconds ---" % (time.time() - start_time))

--- 119.20522475242615 seconds ---


In [4]:
start_time = time.time()
topomap_ml_times = TopoMapMlPack(X_times)
proj_topomap_ml_times = topomap_ml_times.run()
print("--- %s seconds ---" % (time.time() - start_time))

--- 105.13412404060364 seconds ---


In [5]:
def plot_topomap_comparison_plotly(proj_original, proj_new,
                                  hiertopomap=None):
    
    fig = make_subplots(rows=1, cols=2,
                        specs=[[{'type': 'xy'},
                                {'type': 'xy'}]],
                        subplot_titles=('Original Projection - TopoMap',
                                        'New Projection - Hierarchical TopoMap'))
    
    fig.add_trace(
        go.Scatter(x=proj_original[:,0], 
                   y=proj_original[:,1],
                   mode='markers',
                   opacity=0.5,
                   marker=dict(
                    color='Blue',
                    size=5,
                   )
                   ),
        row=1, col=1
    )

    if not hiertopomap is None:
        hulls = hiertopomap.get_components_hull()
        for j, sub in enumerate(hiertopomap.subsets):
            if len(sub) > 10:
                comp_ids = list(hiertopomap.subsets[j])
                points_ids = [comp_ids[i] for i in hulls[j].vertices]
                points = list(hiertopomap.projections[points_ids,:])
                points.append(points[0])
                xs, ys = zip(*points)

                fig.add_trace(go.Scatter(x=xs, y=ys,
                                fill='toself', fillcolor = 'rgba(255, 252, 187, 0.5)',
                                line_color='rgb(255, 252, 187)',
                                text=f'Component {j}'
                                ),
                            row=1, col=2)
                
    fig.add_trace(
        go.Scatter(x=proj_new[:,0], 
                   y=proj_new[:,1],
                   mode='markers',
                   opacity=0.5,
                   marker=dict(
                    color='Blue',
                    size=5,
                   )
                   ),
        row=1, col=2
    )

    fig.update_layout(showlegend=False,
                      margin = dict(t=50, l=25, r=25, b=25),
                      height=600,
                      width=1000)

    return fig

In [7]:
fig = plot_topomap_comparison_plotly(proj_topomap_times, proj_topomap_ml_times)
fig.update_layout(height=600, width=1000, title='Times Square Dataset', 
                  xaxis=dict(range=[-400, 800],showticklabels=True), 
                  yaxis=dict(range=[20400,21600],showticklabels=True),
                  xaxis2=dict(range=[2900, 4000],showticklabels=True), 
                  yaxis2=dict(range=[20400,21500],showticklabels=True))
fig.show()